# Part I. ETL Pipeline for Pre-Processing the Files

## Import Python packages 

In [ ]:
import pandas as pd
import os
import glob
import csv
import time
from src.database import (
    get_cs_cluster,
    get_cs_session,
    close_cs_session,
    shutdown_cs_cluster,
    insert_cs_rows,
    create_cs_keyspace,
    create_cs_table,
    drop_cs_table,
    drop_cs_keyspace,
    set_cs_keyspace,
)

## Creating list of filepaths to process original event csv data files

In [ ]:
data_dir = os.getcwd() + "/event_data"
data_files = glob.glob(os.path.join(data_dir, "*"))
print(f"Number of data files: {len(data_files)}")

## Processing the files

In [ ]:
full_data_rows_list = []
for f in data_files:
    with open(f, "r", encoding="utf8", newline="") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # skip header
        for line in csvreader:
            full_data_rows_list.append(line)

print(
    f"Full data: \t#rows = {len(full_data_rows_list)}, "
    f"\tsize = {full_data_rows_list.__sizeof__() / 1000:.2f} kB (size of list in cache)"
)

# creating a smaller event data csv file
csv.register_dialect("myDialect", quoting=csv.QUOTE_ALL, skipinitialspace=True)
datafile_path = "event_datafile_new.csv"
datafile_header = [
    "artistName",
    "userFirstName",
    "userGender",
    "itemInSession",
    "userLastName",
    "songLength",
    "level",
    "location",
    "sessionId",
    "songTitle",
    "userId",
]
datafile_header_map = {name: idx for idx, name in enumerate(datafile_header)}
with open(datafile_path, "w", encoding="utf8", newline="") as f:
    writer = csv.writer(f, dialect="myDialect")
    writer.writerow(datafile_header)
    for row in full_data_rows_list:
        if row[0] == "":
            continue
        writer.writerow(
            (
                row[0],
                row[2],
                row[3],
                row[4],
                row[5],
                row[6],
                row[7],
                row[8],
                row[12],
                row[13],
                row[16],
            )
        )

with open(datafile_path, "r", encoding="utf8") as f:
    num_filtered_rows = sum(1 for line in f) - 1  # subtract 1 for header
    print(
        f"Filtered data: \t#rows = {num_filtered_rows}, "
        f"\tsize = {os.path.getsize(datafile_path) / 1000:.2f} kB (size of file on disk)\n"
    )

# Part II. Complete the Apache Cassandra coding portion of your project.

Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

![](images/image_event_datafile_new.jpg)

## Apache Cassandra Cluster

### Creating a Cluster, keyspace and a session

In [ ]:
cluster = get_cs_cluster()
session = get_cs_session(cluster)
keyspace_name = "my_keyspace"
drop_cs_keyspace(session, keyspace_name)  # drop any existing keyspace (nice for reruns)
create_cs_keyspace(session, keyspace_name)
set_cs_keyspace(session, keyspace_name)  # set the keyspace to the one we just created
tables = set()

## Queries
Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Query 1

**Assignment**: Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

**Query**:
```sql
SELECT artistName, songTitle, songLength 
FROM music_history_by_session
WHERE sessionId = 338 AND itemInSession = 4;
```

**Why this table fits the query**
- Partition on `sessionId` keeps all rows for a session together.
- Clustering on `itemInSession` preserves playback order.
- Adding `userId` to the clustering key prevents collisions when `sessionId` is not globally unique.


#### Creating the table

In [ ]:
music_hist_session = "music_history_by_session"
tables.add(music_hist_session)
drop_cs_table(session, music_hist_session)  # drop the table if it exists
music_hist_session_cnames = [
    "userId",
    "sessionId",
    "itemInSession",
    "artistName",
    "songTitle",
    "songLength",
]
music_hist_session_column_types = [
    "int",
    "int",
    "int",
    "text",
    "text",
    "float",
]
music_hist_session_columns = dict(
    zip(music_hist_session_cnames, music_hist_session_column_types)
)
primary_keys = ["sessionId", "itemInSession", "userId"]
create_cs_table(
    session,
    music_hist_session,
    music_hist_session_columns,
    primary_keys,
)

#### Inserting data

In [ ]:
time.sleep(2)  # wait for table to be created in (local) Cassandra
i = 0
with open(datafile_path, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        row = (
            int(line[datafile_header_map["userId"]]),
            int(line[datafile_header_map["sessionId"]]),
            int(line[datafile_header_map["itemInSession"]]),
            line[datafile_header_map["artistName"]],
            line[datafile_header_map["songTitle"]],
            float(line[datafile_header_map["songLength"]]),
        )
        progress_msg = f"Progress: Inserting row {i}/{num_filtered_rows} -> {row}"
        print(progress_msg, end="\r")
        insert_cs_rows(session, music_hist_session, music_hist_session_cnames, [row])
        print(" " * len(progress_msg), end="\r")  # clear line
        i += 1

#### Verification of data

In [ ]:
query = """
SELECT artistName, songTitle, songLength 
FROM music_history_by_session 
WHERE sessionId = 338 AND itemInSession = 4;
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = [(row.artistname, row.songtitle, row.songlength) for row in rows]
df = pd.DataFrame(data, columns=["Artist", "Song", "Length (s)"])
print(df.to_string(index=False))

### Query 2

**Assignment**: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

**Query**:
```sql
SELECT artistName, songTitle, userFirstName, userLastName 
FROM music_history_by_user
WHERE userId = 10 AND sessionId = 182 
ORDER BY itemInSession;
```

**Why this table fits the query**
- Composite partition key `(userId, sessionId)` keeps each user-session together and prevents wide partitions across many sessions for one user.
- Clustering on `itemInSession` preserves order within the session and keeps rows unique.


#### Creating the table

In [ ]:
music_hist_user = "music_history_by_user"
tables.add(music_hist_user)
drop_cs_table(session, music_hist_user)  # drop the table if it exists
music_hist_user_cnames = [
    "userId",
    "sessionId",
    "itemInSession",
    "artistName",
    "songTitle",
    "userFirstName",
    "userLastName",
]
music_hist_user_column_types = [
    "int",
    "int",
    "int",
    "text",
    "text",
    "text",
    "text",
]
music_hist_user_columns = dict(
    zip(music_hist_user_cnames, music_hist_user_column_types)
)
primary_keys = ["(userId, sessionId)", "itemInSession"]
create_cs_table(
    session,
    music_hist_user,
    music_hist_user_columns,
    primary_keys,
)

#### Inserting data

In [ ]:
time.sleep(2)  # wait for table to be created in (local) Cassandra
i = 0
with open(datafile_path, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        row = (
            int(line[datafile_header_map["userId"]]),
            int(line[datafile_header_map["sessionId"]]),
            int(line[datafile_header_map["itemInSession"]]),
            line[datafile_header_map["artistName"]],
            line[datafile_header_map["songTitle"]],
            line[datafile_header_map["userFirstName"]],
            line[datafile_header_map["userLastName"]],
        )
        progress_msg = f"Progress: Inserting row {i}/{num_filtered_rows} -> {row}"
        print(progress_msg, end="\r")
        insert_cs_rows(session, music_hist_user, music_hist_user_cnames, [row])
        print(" " * len(progress_msg), end="\r")  # clear line
        i += 1

#### Verification of data

In [ ]:
query = """
SELECT artistName, songTitle, userFirstName, userLastName 
FROM music_history_by_user
WHERE userId = 10 AND sessionId = 182 
ORDER BY itemInSession;
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = [
    (row.artistname, row.songtitle, row.userfirstname, row.userlastname) for row in rows
]
df = pd.DataFrame(data, columns=["Artist", "Song", "First Name", "Last Name"])
print(df.to_string(index=False))

### Query 3

**Assignment**: Give every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

**Query**:
```sql
SELECT userFirstName, userLastName 
FROM music_history_by_song
WHERE songTitle = 'All Hands Against His Own';
```

**Why this table fits the query**
- Partition on `songTitle` aligns with the `WHERE songTitle = ...` filter.
- Clustering on `userId`, `sessionId`, and `itemInSession` keeps multiple listens unique across users and sessions while allowing repeated plays of the same song.

#### Creating the table

In [ ]:
music_hist_song = "music_history_by_song"
tables.add(music_hist_song)
drop_cs_table(session, music_hist_song)  # drop the table if it exists
music_hist_song_cnames = [
    "userId",
    "sessionId",
    "itemInSession",
    "songTitle",
    "userFirstName",
    "userLastName",
]
music_hist_song_column_types = [
    "int",
    "int",
    "int",
    "text",
    "text",
    "text",
]
music_hist_song_columns = dict(
    zip(music_hist_song_cnames, music_hist_song_column_types)
)
primary_keys = ["songTitle", "userId", "sessionId", "itemInSession"]
create_cs_table(
    session,
    music_hist_song,
    music_hist_song_columns,
    primary_keys,
)

#### Inserting data

In [ ]:
time.sleep(2)  # wait for tables to be created in (local) Cassandra
i = 0
with open(datafile_path, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        row = (
            int(line[datafile_header_map["userId"]]),
            int(line[datafile_header_map["sessionId"]]),
            int(line[datafile_header_map["itemInSession"]]),
            line[datafile_header_map["songTitle"]],
            line[datafile_header_map["userFirstName"]],
            line[datafile_header_map["userLastName"]],
        )
        progress_msg = f"Progress: Inserting row {i}/{num_filtered_rows} -> {row}"
        print(progress_msg, end="\r")
        insert_cs_rows(session, music_hist_song, music_hist_song_cnames, [row])
        print(" " * len(progress_msg), end="\r")  # clear line
        i += 1

#### Verification of data

In [ ]:
query = """
SELECT userFirstName, userLastName 
FROM music_history_by_song
WHERE songTitle = 'All Hands Against His Own';
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = [(row.userfirstname, row.userlastname) for row in rows]
df = pd.DataFrame(data, columns=["First Name", "Last Name"])
print(df.to_string(index=False))

### Drop the tables before closing out the sessions

In [ ]:
for table in tables:
    drop_cs_table(session, table)

### Close the session and cluster connection¶

In [ ]:
close_cs_session(session)
shutdown_cs_cluster(cluster)